## AutoLiftout Figures


In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem.detection import detection

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os, glob

from fibsem.structures import FibsemImage

import os
from copy import deepcopy
from pathlib import Path

import napari
import napari.utils.notifications
import numpy as np
import tifffile as tff
from PyQt5 import QtWidgets

from fibsem.detection import detection
from fibsem.detection import utils as det_utils
from fibsem.detection.detection import DetectedFeatures
from fibsem.segmentation import model as fibsem_model
from fibsem.segmentation.model import load_model
from fibsem.structures import (
    BeamType,
    FibsemImage,
    Point,
)
from PyQt5.QtCore import pyqtSignal
from fibsem.ui.qtdesigner_files import FibsemDetectionWidget
import logging

CHECKPOINT_PATH = os.path.join(os.path.dirname(fibsem_model.__file__), "models", "model4.pt")

In [ ]:


filenames = ["/home/patrick/github/autoliftout/docs/img/01-subtle-bear/tif/ref_liftout_needle_eb.tif",
             "/home/patrick/github/autoliftout/docs/img/01-subtle-bear/tif/ref_liftout_needle_ib.tif", 
             "/home/patrick/github/autoliftout/docs/img/01-subtle-bear/tif/landing_needle_ready_position_ib.tif", 
             "/home/patrick/github/autoliftout/docs/img/01-subtle-bear/tif/landing_lamella_needle_removal_low_res_ib.tif"]




checkpoint = "openfibsem-baseline-34.pt"
encoder="resnet34"
num_classes = 3
model = load_model(checkpoint=checkpoint, encoder=encoder, nc=num_classes)



feature_list = [[detection.NeedleTip(), detection.LamellaLeftEdge()],
                [detection.NeedleTip(), detection.LamellaLeftEdge()],
                [detection.LamellaRightEdge(), detection.LandingPost()], 
                [detection.NeedleTip(), detection.LamellaLeftEdge()]]


for fname, features in zip(filenames, feature_list):
    image = FibsemImage.load(fname)

    # detect features
    pixelsize = image.metadata.pixel_size.x if image.metadata is not None else 25e-9
    det = detection.detect_features(
        deepcopy(image.data), model, features=features, pixelsize=pixelsize
    )

    fig, ax = plt.subplots(1, 1, figsize=(7, 5))

    ax.imshow(det.image, cmap="gray")
    ax.imshow(det.rgb, alpha=0.3)
    # ax.set_title("Prediction")
    ax.axis("off")
    for f in det.features:
        
        if isinstance(f, (detection.LamellaCentre, 
                          detection.LamellaLeftEdge, 
                          detection.LamellaRightEdge)):
            f.color = "red"

        ax.plot(f.px.x, f.px.y, 
                    "o",  color=f.color, 
                    markersize=5, markeredgecolor="w", 
                    label=f.name)
    ax.legend(loc="best")

    if len(det.features) == 2:
        # plot white line between features
        ax.plot([det.features[0].px.x, det.features[1].px.x],
                    [det.features[0].px.y, det.features[1].px.y], 
                    color="w", linestyle="--")

    # save fig dpi=300
    save_path = os.path.join(os.getcwd(), "figures")
    os.makedirs(save_path, exist_ok=True)
    basename = os.path.basename(fname).replace(".tif", ".png")
    plt.savefig(os.path.join(save_path, basename), dpi=300, bbox_inches="tight")

    plt.show()


In [ ]:


PATH = "/home/patrick/github/fibsem/scratch/figure/liftout/raw/undercut/waffle/*.tif"

filenames = sorted(glob.glob(PATH, recursive=True))


checkpoint = "autolamella-05-34.pt"
encoder="resnet34"
num_classes = 3
model = load_model(checkpoint=checkpoint, encoder=encoder, nc=num_classes)


feature_list = [[detection.LamellaCentre()],
            [detection.LamellaCentre()],
            [detection.LamellaTopEdge()],
            [detection.LamellaTopEdge()]
]

for i, (fname, features) in enumerate(zip(filenames, feature_list)):
    image = FibsemImage.load(fname)

    # detect features
    pixelsize = image.metadata.pixel_size.x if image.metadata is not None else 25e-9
    det = detection.detect_features(
        deepcopy(image.data), model, features=features, pixelsize=pixelsize)

    fig, ax = plt.subplots(1, 1, figsize=(7, 5))

    det.rgb[:, :, 1] =  0

    ax.imshow(det.image, cmap="gray")
    # ax.imshow(det.rgb, alpha=0.3)
    # if point:
    #     ax.plot(point.x, point.y, "o", color="w", markersize=5, markeredgecolor="w", label="Point")
    ax.axis("off")
    for f in det.features:
        
        if isinstance(f, (detection.LamellaCentre, 
                          detection.LamellaLeftEdge, 
                          detection.LamellaRightEdge)):
            f.color = "red"

        ax.plot(f.px.x, f.px.y, 
                    "o",  color=f.color, 
                    markersize=5, markeredgecolor="w", 
                    label=f.name)
    ax.legend(loc="best")

    if len(det.features) == 2:
        # plot white line between features
        ax.plot([det.features[0].px.x, det.features[1].px.x],
                    [det.features[0].px.y, det.features[1].px.y], 
                    color="w", linestyle="--")

    # plot centre of image
    if i in [0, 1]:
        ax.plot(det.image.shape[1]//2, det.image.shape[0]//2, "+", color="white", markersize=10)

    # save fig dpi=300
    save_path = os.path.join(os.getcwd(), "figures")
    os.makedirs(save_path, exist_ok=True)
    basename = os.path.basename(fname).replace(".tif", ".png")
    plt.savefig(os.path.join(save_path, basename), dpi=300, bbox_inches="tight")

    plt.show()

In [ ]:

PATH = "/home/patrick/github/fibsem/scratch/figure/liftout/raw/undercut/autoliftout/*.tif"

filenames = sorted(glob.glob(PATH, recursive=True))


checkpoint = "autoliftout-waffle-01-34.pt"
encoder="resnet34"
num_classes = 3
model = load_model(checkpoint=checkpoint, encoder=encoder, nc=num_classes)


feature_list = [[detection.LamellaCentre()],
            [detection.LamellaCentre()],
            [detection.LamellaTopEdge()],
            [detection.LamellaTopEdge()]
]


for i, (fname, features) in enumerate(zip(filenames, feature_list)):
    image = FibsemImage.load(fname)
    
    # detect features
    pixelsize = image.metadata.pixel_size.x if image.metadata is not None else 25e-9
    det = detection.detect_features(
        deepcopy(image.data), model, features=features, pixelsize=pixelsize,
    )

    fig, ax = plt.subplots(1, 1, figsize=(7, 5))

    det.rgb[:, :, 1] =  0


    ax.imshow(det.image, cmap="gray")
    # ax.imshow(det.rgb, alpha=0.3)
    ax.axis("off")
    for f in det.features:
        
        if isinstance(f, (detection.LamellaCentre, 
                          detection.LamellaLeftEdge, 
                          detection.LamellaRightEdge)):
            f.color = "red"

        ax.plot(f.px.x, f.px.y, 
                    "o",  color=f.color, 
                    markersize=5, markeredgecolor="w", 
                    label=f.name)
    ax.legend(loc="best")

    if len(det.features) == 2:
        # plot white line between features
        ax.plot([det.features[0].px.x, det.features[1].px.x],
                    [det.features[0].px.y, det.features[1].px.y], 
                    color="w", linestyle="--")

    # plot centre of image
    if i in [0, 1]:
        ax.plot(det.image.shape[1]//2, det.image.shape[0]//2, "+", color="white", markersize=10)


    # save fig dpi=300
    save_path = os.path.join(os.getcwd(), "figures")
    os.makedirs(save_path, exist_ok=True)
    basename = os.path.basename(fname).replace(".tif", ".png")
    plt.savefig(os.path.join(save_path, basename), dpi=300, bbox_inches="tight")

    plt.show()

In [ ]:


filenames = ["/home/patrick/github/autoliftout/docs/img/01-subtle-bear/tif/ref_jcut_low_res_eb.tif",
             "/home/patrick/github/autoliftout/docs/img/01-subtle-bear/tif/ref_jcut_low_res_ib.tif", 
             "/home/patrick/github/autoliftout/docs/img/01-subtle-bear/tif/ref_jcut_high_res_eb.tif", 
             "/home/patrick/github/autoliftout/docs/img/01-subtle-bear/tif/ref_jcut_high_res_ib.tif"]




checkpoint = "openfibsem-baseline-34.pt"
encoder="resnet34"
num_classes = 3
model = load_model(checkpoint=checkpoint, encoder=encoder, nc=num_classes)



feature_list = [[detection.LamellaCentre()],
            [detection.LamellaCentre()],
            [detection.LamellaCentre()],
            [detection.LamellaTopEdge()]
]


for fname, features in zip(filenames, feature_list):
    image = FibsemImage.load(fname)

    if "ref_jcut_high_res_ib.tif" in fname:
        point = Point(x=1200, y=250)
        # print(point)
    else:
        point = None
    # detect features
    pixelsize = image.metadata.pixel_size.x if image.metadata is not None else 25e-9
    det = detection.detect_features(
        deepcopy(image.data), model, features=features, pixelsize=pixelsize, point = point
    )

    fig, ax = plt.subplots(1, 1, figsize=(7, 5))

    det.rgb[:, :, 1] =  0

    ax.imshow(det.image, cmap="gray")
    ax.imshow(det.rgb, alpha=0.3)

    ax.axis("off")
    for f in det.features:
        
        if isinstance(f, (detection.LamellaCentre, 
                          detection.LamellaLeftEdge, 
                          detection.LamellaRightEdge)):
            f.color = "red"

        ax.plot(f.px.x, f.px.y, 
                    "o",  color=f.color, 
                    markersize=5, markeredgecolor="w", 
                    label=f.name)
    ax.legend(loc="best")

    if len(det.features) == 2:
        # plot white line between features
        ax.plot([det.features[0].px.x, det.features[1].px.x],
                    [det.features[0].px.y, det.features[1].px.y], 
                    color="w", linestyle="--")

    # save fig dpi=300
    save_path = os.path.join(os.getcwd(), "figures")
    os.makedirs(save_path, exist_ok=True)
    basename = os.path.basename(fname).replace(".tif", ".png")
    plt.savefig(os.path.join(save_path, basename), dpi=300, bbox_inches="tight")

    plt.show()


In [ ]:
liftout_start = "/home/patrick/github/autoliftout/docs/img/01-subtle-bear/tif/needle_liftout_start_position_eb.tif"
landing_ready = "/home/patrick/github/autoliftout/docs/img/01-subtle-bear/tif/landing_needle_ready_position_ib.tif"
landing_finish = "/home/patrick/github/autoliftout/docs/img/01-subtle-bear/tif/landing_lamella_needle_removal_low_res_ib.tif"


weld_exposure = "/home/patrick/github/autoliftout/docs/img/01-subtle-bear/tif/ref_landing_lamella_high_res_ib.tif"
weld_spot = ""

checkpoint = str(CHECKPOINT_PATH)
encoder="resnet34"
num_classes = 3
model = load_model(checkpoint=checkpoint, encoder=encoder, nc=num_classes)


filenames = [liftout_start, landing_ready, landing_finish]

feature_list = [[detection.NeedleTip(), detection.LamellaLeftEdge()],
[detection.LamellaRightEdge(), detection.LandingPost()], 
[detection.NeedleTip(), detection.LamellaLeftEdge()]]


for fname, features in zip(filenames, feature_list):
    image = FibsemImage.load(fname)

    # detect features
    pixelsize = image.metadata.pixel_size.x if image.metadata is not None else 25e-9
    det = detection.detect_features(
        deepcopy(image.data), model, features=features, pixelsize=pixelsize
    )

    fig, ax = plt.subplots(1, 1, figsize=(7, 5))

    ax.imshow(det.image, cmap="gray")
    ax.imshow(det.rgb, alpha=0.3)
    # ax.set_title("Prediction")
    ax.axis("off")
    for f in det.features:
        
        if isinstance(f, (detection.LamellaCentre, 
                          detection.LamellaLeftEdge, 
                          detection.LamellaRightEdge)):
            f.color = "red"

        ax.plot(f.px.x, f.px.y, 
                    "o",  color=f.color, 
                    markersize=5, markeredgecolor="w", 
                    label=f.name)
    ax.legend(loc="best")

    if len(det.features) == 2:
        # plot white line between features
        ax.plot([det.features[0].px.x, det.features[1].px.x],
                    [det.features[0].px.y, det.features[1].px.y], 
                    color="w", linestyle="--")

    # save fig dpi=300
    save_path = os.path.join(os.getcwd(), "figures")
    os.makedirs(save_path, exist_ok=True)
    basename = os.path.basename(fname).replace(".tif", ".png")
    plt.savefig(os.path.join(save_path, basename), dpi=300, bbox_inches="tight")

    plt.show()


In [ ]:
# convert to png
weld_exposure = "/home/patrick/github/autoliftout/docs/img/01-subtle-bear/tif/ref_landing_lamella_high_res_ib.tif"
weld_spot = "/home/patrick/github/fibsem/scratch/figure/liftout/ref_landing_lamella_high_res_ib.tif"

outnames = ["weld_exposure.png", "weld_spot.png"]
filenames = [weld_exposure, weld_spot]
for fname, outname in zip(filenames, outnames):

    image = FibsemImage.load(fname)
    image.data = image.data.astype(np.float32)

    # save as png
    save_path = os.path.join(os.getcwd(), "figures")
    os.makedirs(save_path, exist_ok=True)
    basename = outname
    plt.imsave(os.path.join(save_path, basename), image.data, cmap="gray")


In [ ]:
# celgans: /home/patrick/github/autoliftout/docs/img/01-gentle-dog
# dm-embryo: /home/patrick/github/autoliftout/docs/img/01-subtle-bear

no_glow_path = "/home/patrick/github/autoliftout/docs/img/01-gentle-dog"
glow_path = "/home/patrick/github/autoliftout/liftout/tools/figures/contact/2"



no_glow_files = sorted(glob.glob(os.path.join(no_glow_path, "*contact_brightness*.tif")))
glow_files = sorted(glob.glob(os.path.join(glow_path, "*contact_brightness*.tif")))

from pprint import pprint
# pprint(no_glow_files)
# pprint(glow_files)

N_IMAGES = 6# max(len(no_glow_files),len(glow_files))
filenames = list(zip(no_glow_files[-N_IMAGES:], glow_files[-N_IMAGES:]))

# fig, ax = plt.subplots(3, len(filenames), figsize=(15, 5))

fig = plt.figure(figsize=(25, 10))
gs = fig.add_gridspec(3, len(filenames))

noglow_brightness = []
glow_brightness = []

for i, (f1, f2) in enumerate(filenames):
    
    print(f1, f2)

    ax1 = fig.add_subplot(gs[0, i])
    ax2 = fig.add_subplot(gs[1, i])
    # plot
    img1= tff.imread(f1)
    img2 = tff.imread(f2)

    # get brightness
    noglow_brightness.append(img1.mean())
    glow_brightness.append(img2.mean())

    ax1.imshow(img1, cmap="gray")
    ax2.imshow(img2, cmap="gray")

    # axes off
    ax1.axis("off")
    ax2.axis("off")

ax3 = fig.add_subplot(gs[2, :])

# plot brightness across all columns of subplot
ax3.plot(range(len(noglow_brightness)), noglow_brightness, "blue", label="C.elegans")
ax3.plot(range(len(glow_brightness)), glow_brightness, "red", label="D.melanogasters")
ax3.set_xlabel("Step")
ax3.set_ylabel("Average Intensity")
ax3.legend(loc="best", fontsize=14)


ax3.text(0.5, 150, "No Contact", fontsize=12, verticalalignment='top', horizontalalignment='center')
ax3.text(2.5, 150, "Partial Contact", fontsize=12, verticalalignment='top', horizontalalignment='center')
ax3.text(4.5, 150, "Full Contact", fontsize=12, verticalalignment='top', horizontalalignment='center')

# vertical line
ax3.axvline(x=1.5, color="gray", linestyle="--")
ax3.axvline(x=3.5, color="gray", linestyle="--")


# no spacing
plt.subplots_adjust(wspace=0, hspace=0.01)

# save figure dpi = 300
save_path = os.path.join(os.getcwd(), "figures")
os.makedirs(save_path, exist_ok=True)
basename = "contact_brightness.png"
plt.savefig(os.path.join(save_path, basename), dpi=300, bbox_inches="tight")

plt.show()

## Run Statistics


In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("statistics.csv")

# convert date column to datetime
df["date"] = pd.to_datetime(df["Date"], format="%d/%m/%Y")

# drop Date column
df = df.drop("Date", axis=1)

# drop rows with Attempted = 0
df = df[df["Attempted"] != 0]

#rename Succesful to Successful
df = df.rename(columns={"Succesful": "Successful"})
# calculate success rate
df["Success"] = df["Successful"] / df["Attempted"]

# sort df by date
df = df.sort_values(by="date")

In [ ]:

# calculate months since min
# get min year
min_year = df["date"].min().year
# get min month in min year
min_month = df[df["date"].dt.year == min_year]["date"].min().month


# filter to

# calculate months since min
df["months_since_min"] = (df["date"].dt.year - min_year) * 12 + (df["date"].dt.month - min_month)

df_grouped = df.groupby(["Sample", "months_since_min"]).sum().reset_index()

# drop all columns except Sample, months_since_min, Attempted, Successful
df_grouped = df_grouped[["Sample", "months_since_min", "Attempted", "Successful"]]

# calculate success rate
df_grouped["Success"] = df_grouped["Successful"] / df_grouped["Attempted"]

# sort by months since min
df_grouped = df_grouped.sort_values(by="months_since_min")

display(df_grouped)

# plot success as  bar chart, color by sample with plotly express
import plotly.express as px

fig = px.bar(df_grouped, x="months_since_min", y="Success", color="Sample", barmode="group")


# set xlabel as Months Since First Attempt
fig.update_layout(title="Success Rate vs Month", xaxis_title="Months Since First Attempt")

fig.show()


In [ ]:
display(df)

# drop rows with Note = No Attempt
# df = df[df["Note"] != "No Attempt"]


# drop dates before Dec 2022
df2 = df[df["date"] >= pd.to_datetime("2022-12-01")]



# calculate overal success rate
# success_rate = df["Successful"].sum() / df["Attempted"].sum()
succesful = df2["Successful"].sum()
attempted = df2["Attempted"].sum()

print(succesful, attempted)

print(succesful/attempted)

print(succesful-9, attempted-9)

print((succesful-9)/(attempted-9))


In [ ]:
# drop dates before Nov 2022
# df = df[df["date"] >= "2022-12-01"]

# plot df, success rate vs date, plotly express
fig = px.scatter(df, x="date", y="Success", color="Sample")
# set title, success rate
fig.update_layout(title="Success Rate vs Date", yaxis_title="Success Rate")
fig.show()

In [ ]:
# plot attempted vs date, plotly express
fig = px.scatter(df, x="date", y="Attempted", color="Sample")
# set title, success rate
fig.update_layout(title="Attempted vs Date", yaxis_title="Attempted")
fig.show()

In [ ]:
# sum attempted and successful by month

import numpy as np


# fill na with empty string
df["Note"] = df["Note"].fillna("")

# drop if "Arc" in Note
df2= df[~df["Note"].str.contains("Arc")]


# group by Sample
df2 = df2.groupby("Sample").sum().reset_index()


# calculate success rate
df2["Success"] = df2["Successful"] / df2["Attempted"]

# sort by Success
display(df2)


In [ ]:
display(df)

# group by date, sum attempted, successful
df_grouped = df.groupby("date").sum().reset_index()


# average attempted
avg_attempts = df_grouped["Attempted"].mean()

print(f"Average Attempts: {avg_attempts}")







In [ ]:
# get note column
notes = df["Note"]

# get all notes that are not nan
notes = notes[~notes.isna()]

# to list
notes = notes.tolist()

from pprint import pprint

# join all notes into one string

notes = " ".join(notes)
# drop ,
notes = notes.replace(",", "")
# drop .
notes = notes.replace(".", "")
# drop '
notes = notes.replace("'", "")

# split by space
notes = notes.split(" ")

# drop "Lucky", "Chow"
notes = [note for note in notes if note not in ["Lucky", "Chow"]]

# Replace "JCut" "Jcut" with "Undercut"
notes = [note if note not in ["JCut", "Jcut"] else "Undercut" for note in notes]

# drop 'No'
notes = [note for note in notes if note != "No"]
#  drop Attempt
notes = [note for note in notes if note != "Attempt"]
# count the number of times each word appears
from collections import Counter
notes = Counter(notes)
pprint(notes)



# plot with plotly express, color by key , pie chart

import plotly.express as px

fig = px.pie(
    values=list(notes.values()), names=list(notes.keys()), title="Notes", color=list(notes.keys())
)

#set title to Failure Modes
fig.update_layout(title="Failure Modes")
fig.show()

## Model Statistics

Show different versions of the model performance on masks, and detections


#### Milling Protocols Optimisation

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

import os
import glob

from pprint import pprint

PATH = "/home/patrick/github/data/EXPERIMENTS/HANNAH-WAFFLE-01-23082023"


pprint(filenames)


In [ ]:
df = pd.read_csv(os.path.join(PATH, "history.csv"))

df_grouped = df.groupby("stage").mean().reset_index()

display(df_grouped)

# convert duration into minutes
df_grouped["duration"] = df_grouped["duration"] / 60

# drop stage = Finished, ReadyTrench, ReadyLamella, SetupTrench
df_grouped = df_grouped[~df_grouped["stage"].isin(["Finished", "ReadyTrench", "ReadyLamella", "SetupTrench", "SetupLamella"])]

category_orders = {"stage": ["MillTrench", "MillUndercut", "MillFeatures", "MillRoughCut", "MillRegularCut", "MillPolishingCut"]}

# plot bar chart with plotly express
fig = px.bar(df_grouped, x="stage", y="duration", color="stage", category_orders=category_orders)

fig.update_layout(title="Average Stage Duration", yaxis_title="Duration (min)")



In [ ]:

df_steps = pd.read_csv(os.path.join(PATH, "steps.csv"))

display(df_steps)



# group by stage
df_steps_grouped = df_steps.groupby(["stage", "step"]).mean().reset_index()


# filter to stage = MillTrench, MillUndercut
df_steps_grouped = df_steps_grouped[df_steps_grouped["stage"].isin(["MillTrench", "MillUndercut"])]
display(df_steps_grouped)

# plot as bar chart with plotly express
# fig = px.bar(df_steps_grouped, x="stage", y="duration", color="step", barmode="group", category_orders=category_orders)

# fig.update_layout(title="Average Step Duration", yaxis_title="Duration (min)")



# plot as pie chart 
fig = px.pie(df_steps_grouped, values="duration", names="step", title="Step Duration", color="step", facet_col="stage")
fig.update_layout(title="Average Step Duration Per Stage")

# dont show text if less than 10%
fig.update_traces(textposition='inside', textinfo='percent+label', insidetextorientation='radial', textfont_size=12)

# no legend
fig.update_layout(showlegend=True)

fig.show()
